# Derivation of CCSD theory

This notebook serves as an example of interactive usage of drudge for complex symbolic manipulations in Jupyter notebooks.  Here we can see how the traditional CCD theory can be derived easily.

## Preparatory work

First, we need to set up the Spark environment.  Here we just use parallelization on the local machine.

In [1]:
from pyspark import SparkContext
ctx = SparkContext('local[*]', 'ccsd')

Or we can also use the dummy spark to emulate the Spark environment in a purely serial way.  Note that we need just **one** Spark context.  These two cells should not be both evaluated.

In [ ]:
from dummy_spark import SparkContext
ctx = SparkContext()

With the Spark context, we can construct the drudge specific for this problem.  Then we can define some names that is going to be used frequently.

In [2]:
from sympy import *
from drudge import *

dr = PartHoleDrudge(ctx)
dr.full_simplify = False
p = dr.names

c_ = p.c_
c_dag = p.c_dag
a, b = p.V_dumms[:2]
i, j = p.O_dumms[:2]

In [3]:
p,q,r,s = dr.all_orb_dumms[:4]

## Define Hamiltonian

Most drudge classes (not all) come with an inbuilt general Hamiltonian; but we can also define any specific Hamiltonian.

For defining our new Hamiltonian, we first need to define the matrix elements / integrals

In [4]:
h = IndexedBase('h')
v = IndexedBase('v')

We know that the 1-electron integral will be symmetric (if real) and the 2-electron integral has a 4-fold symmetry

In [5]:
dr.set_symm(h,
    Perm([1,0],IDENT),
    valence=2
)

In [6]:
dr.set_symm(v,
    Perm([1,0,2,3],NEG),
    Perm([0,1,3,2],NEG),
    Perm([2,3,0,1],IDENT),
    valence=4
)

Now the Hamiltonian - we use an einstein summation function `einst`

In [7]:
ham_new = dr.einst(
    h[p,q]*c_dag[p]*c_[q]
    + v[p,q,r,s]*c_dag[p]*c_dag[q]*c_[s]*c_[r]/4
)

In [8]:
ham_new.simplify().merge().display()

<IPython.core.display.Math object>

We can also use the inbuilt Hamiltonian

In [9]:
ham_inbuilt = dr.ham
ham_inbuilt.simplify().display()

<IPython.core.display.Math object>

## Cluster excitation operator

Here, we by using the Einstein summation convention tensor creator, we can just define the cluster operator in a way very similar to how we would writen them down on paper.

In [10]:
t = IndexedBase('t')

clusters = dr.einst(
    t[a, b, i, j] * c_dag[a] * c_dag[b] * c_[j] * c_[i] / 4
)

We can have a peek at the cluster operator.

In [11]:
clusters.display()

<IPython.core.display.Math object>

Now we need tell the system about the symmetry on $t^2$, so that it can be used in simplification.

## Similarity transform of the Hamiltonian

Here we can use a loop to nest the commutation conveniently.  And IPython magic can be used to time the operation.  Note that after the simplification, we explicitly redistribute the terms in the transformed Hamiltonian for better parallel performance in later operations.  Note that `drudge` does not automatically cache the result of tensor computations.  The `cache` method should be called explicitly when a tensor is going to be used multiple times.

In [12]:
%%time

curr = dr.ham
h_bar = dr.ham
for order in range(0, 4):
    curr = (curr | clusters).simplify() / (order + 1)
    curr.cache()
    h_bar += curr
h_bar.repartition(cache=True)

CPU times: user 276 ms, sys: 53.2 ms, total: 329 ms
Wall time: 33.3 s


The transformed Hamiltonian can be very complex.  Instead of reading its terms, we can just have a peek by get a count of the number of terms it contains.

In [13]:
h_bar.n_terms

232

## Working equation derivation

With the similarity transformed Hamiltonian, we are now ready to derive the actual working equations.  First, the energy equation can be derived by taking the vacuum expectation value of the transformed Hamiltonian.

In [14]:
en_eqn = h_bar.eval_fermi_vev().simplify()

We can have a look at its contents to see if it is what we would expect.

In [15]:
en_eqn.display()

<IPython.core.display.Math object>

The working equation for the doubles amplitude can be done in the same way, just it can be slower.

In [16]:
%%time

proj = c_dag[i] * c_dag[j] * c_[b] * c_[a]
t2_eqn = (proj * h_bar).eval_fermi_vev().simplify()

CPU times: user 76 ms, sys: 15.8 ms, total: 91.8 ms
Wall time: 2min 29s


Since the equation can be slightly complex, we can vaguely sort the terms in increasing complexity before display them.

In [17]:
t2_eqn = t2_eqn.sort()
t2_eqn.display()

<IPython.core.display.Math object>

## Working equation optimization

Evaluating the working equation takes a lot of effort.  Outside ``drudge``, a sister package named ``gristmill`` is available for the optimization and automatic code generation for tensor contractions.  To start with, we need to put the working equations into a tensor definitions with external indices and import the ``gristmill`` package.

In [18]:
from gristmill import *

r = IndexedBase('r')
working_eqn = [
    dr.define(Symbol('e'), en_eqn), 
    dr.define(r[a, b, i, j], t2_eqn)
]

We can have an estimation of the FLOP cost without any optimization.

In [19]:
orig_cost = get_flop_cost(working_eqn, leading=True)
init_printing()
orig_cost

Since normally we have far more virtual orbitals than occupied orbitals, we have make the optimization based on this.

In [21]:
%%time
p = dr.names
eval_seq = optimize(
    working_eqn, substs={p.nv: 50, p.no: 10}
)

/projects/guscus/apps-python/drudge/gristmill/build/lib.linux-x86_64-3.6/gristmill/optimize.py:1985: UserWarning: Internal deficiency: summation intermediate may not be fully canonicalized
  'Internal deficiency: '


CPU times: user 3.7 s, sys: 260 ms, total: 3.96 s
Wall time: 24.2 s


Now we can have some inspection of the evaluation sequence.

In [22]:
len(eval_seq)

In [23]:
opt_cost = get_flop_cost(eval_seq, leading=True)
opt_cost

Significant optimization can be seen.  Finally we can verify the correctness of the evaluation sequence.  This step can be very slow.  But it is adviced for mission-critical tasks.

In [ ]:
verify_eval_seq(eval_seq, working_eqn, simplify=True)

Finally, we have have a peek at the details of the intermediates.

In [24]:
for eqn in eval_seq:
    eqn.display(False)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>